#### Example Socrata Module for City of Chicago Data. https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi
* I'll move the client info to a config file later. 
* I don't use the sodapy package, the code is more efficient, but I'm trying to demonstrate that I'm learning the DS_Bootcamp code

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import warnings;
warnings.filterwarnings('ignore');

# import pandas as pd
from sodapy import Socrata

MyAppToken = 'PoLZLhceSUgObx1Disd6WIQS1'

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 MyAppToken,
                 username="stephen.monteiro@northwestern.edu",
                 password="FackeTestPw0rd")

#### Importing Modules & Data
* With more time and a better coder, you could loop through each API call, hence why I define year as a list. Optimistic I'll return to it one day. 

In [2]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint as pp
import os
import csv
year = [2002, 2010, 2018]


In [3]:
fbi_codex = pd.read_csv('fbi_codex.csv')
fbi_index = fbi_codex[['fbi_Crime_type','fbi_code_key','ds_fbi_group']]
fbi_index = fbi_index.dropna()
gent_codex = pd.read_csv('uic_gent_index.csv')
gent_concat = gent_codex["Type"].str.split(" - ", n = 1, expand = True) 
gent_codex["Classification"] = gent_concat[1]
gent_codex["Class_Code"] = gent_concat[0]
gent_fin = gent_codex.rename(columns={"No.": "community_area"})
gent_fin
display(fbi_index.head())
display(gent_fin.head())

FileNotFoundError: File b'fbi_codex.csv' does not exist

#### API Call for Start Year
* Currently using 2002 rather than 2001, because 2001 data lacks 'community_area' for most reported crimes. With more time, you could use loops and conditions on 'latitude' and 'longitude' to fill 'community area'.
* Export the data to CSV so that there is a reference file in case original data source is ever corrupted, or non-responsive to API calls.
* For more on the City of Chicago Crime Data: https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi

* FIGURE OUT: how to remove decimal from neighborhood

In [ ]:
results = client.get("6zsd-86xi", year=2002, limit=5000)
results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame # results_df.head()
start_results = results_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
start_results['community_area'] = pd.to_numeric(start_results['community_area'], errors='coerce')
start_format = start_results.sort_values('community_area')
start_format.to_csv('report_reference_data/start_tfile.csv')    
start_format.head(10)


#### Combining the Crime Data with our Research Indexes (Gentrification & FBI Crime Category)

In [ ]:
b_crimecat_cca_clean = b_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
b_crimecat_cca_group = b_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
crime_reset = b_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


In [ ]:
b_crimecat_cca_clean['Class_Code'] = pd.to_numeric(b_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ng1 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] < 3]
ng2 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 5]
g6 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 6]
combo = [ng1, ng2]
nGentrified = pd.concat(combo)
nGentrified["DS_Subset"] = "Not Gentrified"
g6["DS_Subset"] = "Gentrified"
neighborhodtype_bycrime = [nGentrified, g6]
neighbors = pd.concat(neighborhodtype_bycrime)
neighbors_reset = neighbors.groupby(by=['DS_Subset','ds_fbi_group']).count()
neighbors_dropem = neighbors_reset.reset_index()
neighbors_dropem.drop(columns=['Classification','community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
neighbors_dropem.pivot(index='ds_fbi_group', columns='DS_Subset', values='Class_Code')


In [ ]:
start_group = start_format.groupby(by=['community_area', 'fbi_code']).count() #relevant_results.sort_values('fbi_code'
start_group


In [ ]:
midresults = client.get("6zsd-86xi", year=2010, limit=5000)
midresults_df = pd.DataFrame.from_records(midresults) # Convert to pandas DataFrame # results_df.head()
mid_results = midresults_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
mid_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
mid_results.sort_values(['community_area'])
mid_results.to_csv('report_reference_data/mid_tfile.csv')    
mid_results

In [ ]:
closeresults = client.get("6zsd-86xi", year=2018, limit=5000)
closeresults_df = pd.DataFrame.from_records(closeresults)                    # Convert to pandas DataFrame # results_df.head()
close_results = closeresults_df[['community_area','fbi_code','date','description', 'latitude','longitude','primary_type','year']]
close_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
close_results.to_csv('report_reference_data/close_tfile.csv')    
close_results


In [ ]:
srt_gent = start_results.merge(gent_fin, on='community_area')
srt_gent